In [ ]:
import pickle
import pandas as pd
import numpy as np
import string, os 
import warnings
import h5py
import datetime

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import loadtxt
from tensorflow.keras.models import load_model
#model = load_model('name.h5')

from numpy.random import seed
import tensorflow
#tensorflow.random.set_seed(4)
#seed(20) - original seeds
tensorflow.random.set_seed(int(datetime.datetime.utcnow().timestamp()))
seed(int(datetime.datetime.utcnow().timestamp()))

os.chdir("../quoteAIsData")
names = ["DAVID", "DIEGO", "ERIC", "GWYN", "JETT", "MARS", "MILES", "MILO", "PARSA"]

In [ ]:
def loadToken(name):
    tokenizer = pickle.load(open(name + '_tokenizer.pkl', 'rb'))
    return tokenizer

def loadSequences(name):
    file = open(name+'_sequences.txt', 'r', encoding = "utf-8")
    text = file.read()
    file.close()
    return text

def generateQuotes(name, wordCount, quoteCount):
    tokenizer = loadToken(name)
    lines = loadSequences(name)
    lines = lines.split('\n')
    sequences = tokenizer.texts_to_sequences(lines)
    sequences = np.array(sequences)
    X, y = sequences[:,:-1], sequences[:,-1]
    seq_length = X.shape[1]
    model = load_model(name + '_char_word_model.h5')
    for i in range(0, quoteCount):
        seed_text = lines[np.random.randint(0,len(lines))]
        result = list()
        for _ in range(wordCount):
            # encode the text as integer
            encoded = tokenizer.texts_to_sequences([seed_text])[0]
            # truncate sequences to a fixed length
            encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
            # predict probabilities for each word
            yhat = np.argmax(model.predict(encoded), axis = -1)
            # map predicted word index to word
            out_word = ''
            for word, index in tokenizer.word_index.items():
                if index == yhat:
                    out_word = word
                    break
            # append to input
            seed_text += ' ' + out_word
            result.append(out_word)
        final = ' '.join(result)
        print(f'{name}: "{final}"')
        filename = "0_Results/" + name + ".txt"
        file = open(filename, 'a', encoding = 'utf-8')
        file.write("-" + final + "\n")

In [ ]:
generateQuotes("DAVID", 30, 10)

In [ ]:
generateQuotes("DIEGO", 30, 10)

In [ ]:
generateQuotes("PARSA", 30, 10)

In [ ]:
generateQuotes("MILES", 30, 10)

In [ ]:
generateQuotes("GWYN", 30, 10)

In [ ]:
generateQuotes("JETT", 30, 10)